The goal is to get all the paths from a directory (and its subdirs) in dropbox, then propose for each a new location and a new filename.


# Imports

In [1]:
import os

from utils.filename_reader import create_filename_dict
import json

import sys
import dropbox
from dropbox.files import WriteMode
from dropbox.exceptions import ApiError, AuthError

from tokens import ACCESS_TOKEN

# Add OAuth2 access token here.
# You can generate one for yourself in the App Console.
# See <https://blogs.dropbox.com/developers/2014/05/generate-an-access-token-for-your-own-account/>
TOKEN = ACCESS_TOKEN


# API token

In [2]:
if (len(TOKEN) == 0):
    sys.exit("ERROR: Looks like you didn't add your access token. "
        "Open up backup-and-restore-example.py in a text editor and "
        "paste in your token in line 14.")

    # Create an instance of a Dropbox class, which can make requests to the API.
print("Creating a Dropbox object...")
with dropbox.Dropbox(TOKEN) as dbx:

    # Check that the access token is valid
    try:
        dbx.users_get_current_account()
    except AuthError:
        sys.exit("ERROR: Invalid access token; try re-generating an "
            "access token from the app console on the web.")
    
    

Creating a Dropbox object...


# Get the paths

In [5]:
def get_all_paths(dir):
    all_paths = []
    for entry in dbx.files_list_folder(dir).entries:
        if type(entry) == dropbox.files.FolderMetadata:
            all_paths += get_all_paths(entry.path_display)
        else:
            all_paths.append(entry.path_display)
    return all_paths



In [10]:
l = get_all_paths('/source/REEVOID_PILOT_01_MRI_NIFTI')
for path in l:
    print(path)
print('\n')

l = get_all_paths('/source/REEVOID_PILOT_01_MRI_NIFTI/')
for path in l:
    print(path)
print('\n')

l = get_all_paths('/source/')
for path in l:
    print(path)

/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json


/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json


/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json
/source/Reevoid_Martin_fMRI_interoperability.zip
/source/PhysioLogFiles_fMRI_Martin.zip


# Save jsons with infos and new paths

In [11]:
participants_dict = {
    "REEVOID_001": "sub-01",
    "REEVOID_PILOT_01": "sub-pilot"
}

final_data = {}

for file in l:
    print(file)
    final_data[file] = create_filename_dict(file, participants_dict, sub='sub_pilot',type='func', category='resting_state')
print('done')

with open('file_infos/test.json', 'w') as f:
    json.dump(final_data, f, indent=4)

/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz
/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json
/source/Reevoid_Martin_fMRI_interoperability.zip
/source/PhysioLogFiles_fMRI_Martin.zip
done


## (TO DO) Correct new paths for localizer/other metadata

In [ ]:
# TODO

# Save `paths.txt` and `recap.json`

## `paths.txt`

In [12]:
def write_paths_file(file_infos_path, out_path, old_prefix='', new_prefix=''):

    with open(file_infos_path,'r') as f:
        data = json.load(f)
    
    out_dirs = '/'.join(out_path.split('/')[:-1])
    try:
        os.makedirs(out_dirs)
        with open(out_path,'w') as f:
            for file in data.keys():
                f.write('~' + old_prefix + data[file]['old_path'] + ', ~/' + new_prefix + data[file]['new_path'])
                f.write('\n')
    except:
        with open(out_path,'a') as f:
            for file in data.keys():
                f.write('~' + data[file]['old_path'] + ', ~/' + data[file]['new_path'])
                f.write('\n')



In [ ]:
file_infos_path = 'file_infos/test.json'
out_path = 'paths/test_02.txt'

write_paths_file(file_infos_path, out_path, old_prefix='', new_prefix='')

## `recap.json`

In [13]:
def write_general_recap_file(file_infos_path, out_path, new_prefix=''):

    with open(file_infos_path,'r') as f:
        file_infos = json.load(f)
    
    out_dirs = '/'.join(out_path.split('/')[:-1])
    out_data = {}

    try:
        with open(out_path,'w') as f:
            for file in file_infos.keys():
                type = file_infos[file]['type']
                new_path = new_prefix + file_infos[file]['new_path']
                sub = file_infos[file]['sub']
                if sub not in out_data.keys():
                    out_data[sub] = {}
                if type in out_data[sub].keys():
                    out_data[sub][type].append(new_path)
                else:
                    out_data[sub][type] = [new_path]
            json.dump(out_data, f, indent=4)
    except:
        try:
            os.makedirs(out_dirs)
        finally:
            with open(out_path,'w') as f:
                for file in file_infos.keys():
                    type = file_infos[file]['type']
                    new_path = new_prefix + file_infos[file]['new_path']
                    sub = file_infos[file]['sub']
                    if sub not in out_data.keys():
                        out_data[sub] = {}
                    if type in out_data[sub].keys():
                        out_data[sub][type].append(new_path)
                    else:
                        out_data[sub][type] = [new_path]
                json.dump(out_data, f, indent=4)
        


In [14]:
write_general_recap_file(
    file_infos_path='file_infos/test.json',
    out_path='recaps/recap_01.json',
    new_prefix=''
    )

In [15]:
def merge_general_recaps(input_files, out_path):
    out_data = {}
    for file in input_files:
        with open(file, 'r') as f:
            file_data = json.load(f)
        for sub in file_data.keys():
            if sub not in out_data.keys():
                out_data[sub] = file_data[sub]
            else:
                for file_type in file_data[sub].keys():
                    if file_type not in out_data[sub].keys():
                        out_data[sub][file_type] = file_data[sub][file_type]
                    else:
                        assert type(out_data[sub][file_type]) == list
                        for new_path in file_data[sub][file_type]:
                            if new_path not in out_data[sub][file_type]:
                                out_data[sub][file_type].append(new_path)
    try:
        with open(out_path, 'w') as f:
            json.dump(out_data, f, indent=4)
    except:
        out_dirs = '/'.join(out_path.split('/')[:-1])
        os.makedirs(out_dirs)
        with open(out_path, 'w') as f:
            json.dump(out_data, f, indent=4)
        

In [56]:
merge_general_recaps(
    input_files=[
        'recaps/recap_01.json',
        'recaps/recap_02.json'
    ],
    out_path='recaps/merged_recap_test.json'
)

# Copy and rename old files to their new paths

In [3]:
dbx.files_copy(
    from_path= '/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz',
    to_path='/target/sub_pilot/func/sub_pilot_resting_state_id-7_bold.nii.gz'
)

FileMetadata(client_modified=datetime.datetime(2025, 6, 17, 18, 6, 34), content_hash='e5556a7e2d867e5cd33e1c73bfea5b2561c51ba635f5766a2a895777e2e19d0a', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:KLawZ41SnG8AAAAAAAAATQ', is_downloadable=True, media_info=NOT_SET, name='sub_pilot_resting_state_id-7_bold.nii.gz', parent_shared_folder_id=NOT_SET, path_display='/target/sub_pilot/func/sub_pilot_resting_state_id-7_bold.nii.gz', path_lower='/target/sub_pilot/func/sub_pilot_resting_state_id-7_bold.nii.gz', preview_url=NOT_SET, property_groups=NOT_SET, rev='016410af3178da8000000030e305703', server_modified=datetime.datetime(2025, 10, 13, 14, 19, 4), sharing_info=NOT_SET, size=85415620, symlink_info=NOT_SET)

In [8]:
all_paths = get_all_paths('/source')

print(all_paths)

['/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.nii.gz', '/source/REEVOID_PILOT_01_MRI_NIFTI/Resting_State/REEVOID_PILOT_01_MRI_RestingState_Lumbar_20250528145351_7.json', '/source/Reevoid_Martin_fMRI_interoperability.zip', '/source/PhysioLogFiles_fMRI_Martin.zip']


In [16]:
file_infos_path = 'file_infos/test.json'

with open(file_infos_path, 'r') as f:
    file_infos = json.load(f)

for file in file_infos.keys():
    dbx.files_copy(
    from_path= file_infos[file]["old_path"],
    to_path='/target/' + file_infos[file]["new_path"]
)